# Filtering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cbouy/mols2grid/blob/master/docs/notebooks/filtering.ipynb)

It's possible to integrate the grid with other widgets to complement the searchbar.

In [ ]:
# uncomment and run if you're on Google Colab
# !pip install rdkit mols2grid
# !wget https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/solubility.test.sdf

In [ ]:
from pathlib import Path

from ipywidgets import interact, widgets
from rdkit import RDConfig
from rdkit.Chem import Descriptors

import mols2grid


SDF_FILE = (
    f"{RDConfig.RDDocsDir}/Book/data/solubility.test.sdf"
    if Path(RDConfig.RDDocsDir).is_dir()
    else "solubility.test.sdf"
)

We'll use ipywidgets to add sliders for the molecular weight and the other molecular descriptors, and define a function that queries the internal dataframe using the values in the sliders.

Everytime the sliders are moved, the function is called to filter our grid.

In [ ]:
df = mols2grid.sdf_to_dataframe(SDF_FILE)
# compute some descriptors
df["MolWt"] = df["mol"].apply(Descriptors.ExactMolWt)
df["LogP"] = df["mol"].apply(Descriptors.MolLogP)
df["NumHDonors"] = df["mol"].apply(Descriptors.NumHDonors)
df["NumHAcceptors"] = df["mol"].apply(Descriptors.NumHAcceptors)

In [ ]:
grid = mols2grid.MolGrid(
    df,
    size=(120, 100),
    name="filters",
)
view = grid.display(n_items_per_page=12)


@interact(
    MolWt=widgets.IntRangeSlider(value=[0, 600], min=0, max=600, step=10),
    LogP=widgets.IntRangeSlider(value=[-10, 10], min=-10, max=10, step=1),
    NumHDonors=widgets.IntRangeSlider(value=[0, 20], min=0, max=20, step=1),
    NumHAcceptors=widgets.IntRangeSlider(value=[0, 20], min=0, max=20, step=1),
)
def filter_grid(MolWt, LogP, NumHDonors, NumHAcceptors):
    results = grid.dataframe.query(
        "@MolWt[0] <= MolWt <= @MolWt[1] and "
        "@LogP[0] <= LogP <= @LogP[1] and "
        "@NumHDonors[0] <= NumHDonors <= @NumHDonors[1] and "
        "@NumHAcceptors[0] <= NumHAcceptors <= @NumHAcceptors[1]"
    )
    return grid.filter_by_index(results.index)


view